### Matrix factorization SVD


In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

In [2]:
urm = pd.read_csv("urm.csv", sep=",")

movies = pd.read_csv("procMovies1Hot.csv", sep=",")
movies=movies.drop('Unnamed: 0',axis=1)

In [3]:
R_df=pd.pivot_table(urm, index=['userID'], columns='movieID', values='rating',fill_value =0)
R_df.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
R = R_df.as_matrix()


In [86]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


from surprise import Reader

In [11]:
reader = Reader(line_format = 'user item rating',rating_scale=(1, 5))

data = Dataset.load_from_df(urm[['userID', 'movieID', 'rating']], reader)

In [ ]:
data.head()

In [12]:
algo = SVD(n_epochs = 5)
cross_validate(algo, data, measures=['RMSE', 'MAE'],cv=5)

{'fit_time': (22.72156596183777,
  23.983793020248413,
  22.933403968811035,
  22.987348079681396,
  21.428239822387695),
 'test_mae': array([ 0.7230328 ,  0.72447399,  0.72145824,  0.72419792,  0.72401199]),
 'test_rmse': array([ 0.91136658,  0.91370157,  0.91039409,  0.9137155 ,  0.91241565]),
 'test_time': (4.236482858657837,
  4.0161168575286865,
  3.8889670372009277,
  3.865323066711426,
  3.7116329669952393)}

In [30]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.929207944871
{'n_epochs': 10, 'reg_all': 0.4, 'lr_all': 0.005}


In [76]:
#fit on all dataset
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [77]:
#SVD no bias terms
algo2=SVD(n_epochs = 10,biased=False, reg_all= 0.4, lr_all=0.005)
cross_validate(algo2, data, measures=['RMSE', 'MAE'],cv=5)

{'fit_time': (44.398032903671265,
  41.48491406440735,
  41.30394697189331,
  41.01841497421265,
  41.85299491882324),
 'test_mae': array([ 0.82232916,  0.82177855,  0.82361443,  0.82563705,  0.82469893]),
 'test_rmse': array([ 1.00219555,  1.00103607,  1.00298332,  1.00556924,  1.00410671]),
 'test_time': (3.601630926132202,
  3.4397130012512207,
  2.96026611328125,
  3.336120128631592,
  4.000967979431152)}

In [78]:
algo2.fit(data.build_full_trainset())

In [79]:
P=algo2.pu
Q=algo2.qi

In [80]:
pred=np.matmul(P,Q.T)
preds_df=pd.DataFrame(pred,columns=R_df.columns)

In [81]:
preds_df.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.310980,3.365444,4.013567,3.950597,3.727155,4.075854,4.011095,4.058157,3.589424,4.023413,...,0.214841,0.242608,-0.078438,0.285102,0.387005,0.979069,0.515281,0.547233,0.298773,-0.113329
1,3.866899,3.018382,3.599378,3.545794,3.344644,3.659771,3.598981,3.641875,3.219691,3.609676,...,0.201851,0.145437,-0.058607,0.267832,0.413651,0.867982,0.453629,0.473107,0.304309,-0.063387
2,3.825299,2.981573,3.558215,3.503862,3.306092,3.621115,3.555870,3.599139,3.182327,3.569346,...,0.109889,0.122762,-0.159283,0.261958,0.396350,0.868564,0.522472,0.466283,0.253452,-0.041890
3,3.743086,2.920675,3.483055,3.428902,3.233239,3.535854,3.479731,3.520486,3.115221,3.491364,...,0.217572,0.158802,-0.041626,0.263426,0.405403,0.848903,0.450899,0.473156,0.293924,-0.064230
4,4.402221,3.434382,4.095553,4.030260,3.799493,4.154194,4.090259,4.137474,3.662941,4.104266,...,0.256278,0.186914,-0.049647,0.307229,0.474310,0.997453,0.532724,0.551875,0.346995,-0.075211


In [82]:
#Train test split
trainset, testset = train_test_split(data, test_size=.20)
algo3 = SVD(n_epochs = 5, reg_all= 0.4, lr_all=0.005)

In [32]:
algo3.fit(trainset)
predictions = algo3.test(testset)

In [33]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.9307


0.93067708044745168

In [34]:
from collections import defaultdict
def get_top_n(predictions, n=5):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [84]:
top_n = get_top_n(predictions, n=5)
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

1 [2028, 919, 1287, 1246, 150]
2 [318, 1193, 2858, 1253, 356]
3 [2858, 1304, 3421, 1394, 590]
4 [1196, 2692, 1387, 3702]
5 [2571, 994, 3728, 1243, 1175]
6 [912, 1, 1188, 1028, 3753]
7 [1221, 1997, 1573]
8 [110, 58, 608, 741, 1]
9 [745, 1704, 2692, 47, 994]
10 [260, 898, 969, 1197, 2804]
11 [1198, 1197, 246, 110, 2918]
12 [593, 1221, 3265, 3362, 2616]
13 [50, 1204, 457, 2871, 590]
14 [2692, 2920, 2396, 2997, 2976]
15 [2858, 1196, 3578, 3421, 2268]
16 [2355, 2394, 2683, 2724, 3016]
17 [50, 1193, 2858, 3114, 457]
18 [50, 1193, 1207, 1197, 1242]
19 [720, 593, 858, 1262, 1197]
20 [47, 1527, 70, 1371]
21 [1210, 1274, 3745, 3053]
22 [2324, 1198, 910, 1197, 1225]
23 [1148, 50, 720, 593, 3435]
24 [858, 1198, 1213, 1221, 1225]
25 [2804, 3114, 1610, 3578, 3751]
26 [260, 994, 1307, 3421, 1387]
27 [922, 913, 858, 1207, 2186]
28 [318, 916, 1224, 1197, 2858]
29 [2571, 260, 541, 457, 1387]
30 [527, 2028, 1213, 3911, 2966]
31 [1269, 1230, 3507, 1077, 3836]
32 [50, 2762, 1250, 457, 1617]
33 [50, 2571, 9

430 [933, 356, 1294, 1270, 1680]
431 [260, 3114, 457, 1259, 589]
432 [2396, 1643, 11, 1912, 1680]
433 [1288, 3421, 1394, 539, 3210]
434 [953, 909, 3751, 1948, 2355]
435 [2762, 1221, 1242, 1208, 1674]
436 [2571, 1136, 1196, 110, 457]
437 [527, 1250, 3114, 3681, 3871]
438 [1148, 2028, 1197, 556, 1196]
439 [260, 953, 1276, 1233, 110]
440 [1608, 21, 1573, 3450, 288]
441 [457, 2791, 1097, 1376, 2009]
442 [750, 593, 2571, 953, 1217]
443 [527, 923, 2194, 1097, 555]
444 [904, 1131, 1252, 110, 1617]
445 [2762, 908, 1272, 111, 1259]
446 [1212, 953, 1196, 1199, 2761]
447 [1228, 1231, 1096, 2020, 1022]
448 [2908, 2485, 2581]
449 [1214, 1333, 1997, 3479, 1982]
450 [2028, 1260, 903, 1244, 1952]
451 [2019, 527, 2324, 1189, 1254]
452 [908, 1358, 1704, 1269, 457]
453 [2028, 1172, 2858, 1267, 1136]
454 [1204, 593, 2571, 1213, 1136]
455 [858, 260, 969, 1304, 1196]
456 [1148, 745, 50, 1250, 1198]
457 [2762, 1193, 2028, 1221, 110]
458 [2858, 1222, 3555, 1092, 3927]
459 [2858, 3578, 920, 3948, 3892]
460 [11

859 [2858, 3578, 1261, 3160, 3948]
860 [2366, 1345, 1049, 2747, 3826]
861 [2571, 2918, 2000, 1220, 2797]
862 [1198, 3429, 1219, 1247, 3949]
863 [260, 1610, 2944, 6, 3104]
864 [527, 2731, 908, 1224, 1217]
865 [1172, 1292, 2599, 3510, 780]
866 [3510, 3624, 783, 3785, 3821]
867 [1284, 3504, 3496, 1371]
868 [1387, 3948, 1345, 2519, 3926]
869 [913, 750, 1223, 1198, 2329]
870 [1193, 2706, 94, 3524, 2504]
871 [2762, 3114, 2716, 2355, 2706]
872 [2571, 1449, 1196, 3265, 1256]
873 [1250, 1204, 750, 1262, 3091]
874 [260, 950, 1176, 1777, 3018]
875 [720, 2762, 1219, 3114, 1784]
876 [50, 2571, 1197, 3000, 1233]
877 [318, 527, 1198, 2028, 2858]
878 [741, 29, 3160, 480, 1909]
879 [898, 916, 1221, 965, 3929]
880 [1248, 3114, 1617, 589, 1210]
881 [1148, 593, 910, 1213, 1242]
882 [2010, 1200, 671, 3751, 1097]
883 [3481, 2160, 3753, 1171, 3512]
884 [2762, 858, 2571, 260, 2028]
885 [541, 1213, 1704, 246, 3114]
886 [3578, 1210, 3408, 3328, 3510]
887 [527, 750, 260, 1221, 1245]
888 [2324, 260, 2858, 1240, 5

1315 [527, 1212, 593, 3022, 912]
1316 [913, 2070, 2791, 1305, 2841]
1317 [745, 2324, 1223, 1262, 1284]
1318 [3481, 3555, 2424, 3943]
1319 [1610, 1617, 1201, 3408, 3189]
1320 [593, 1198, 2571, 908, 2804]
1321 [858, 1197, 1299, 608, 919]
1322 [318, 750, 3469, 2804, 1242]
1323 [858, 916, 1224, 1258, 936]
1324 [904, 593, 750, 903, 2858]
1325 [913, 1204, 858, 2571, 898]
1326 [527, 1537, 608, 1280, 1188]
1327 [2028, 3196, 1256, 588, 3448]
1328 [1197, 2662, 2947, 1374, 2949]
1329 [2762, 923, 1197, 3114, 296]
1330 [1212, 913, 260, 919, 1201]
1331 [1207, 1172, 923, 1272, 1358]
1332 [3421, 2396, 3481, 2599, 21]
1333 [318, 1212, 750, 593, 858]
1334 [318, 296, 589, 32, 3160]
1335 [1193, 908, 541, 2858, 1213]
1336 [3114, 1233, 608, 2396, 3006]
1337 [1198, 3114, 919, 3916, 364]
1338 [2571, 608, 1617, 1953, 232]
1339 [2571, 1254, 1197, 1225, 1610]
1340 [527, 913, 1250, 1224, 1254]
1341 [188, 1168, 2026, 1325, 2004]
1342 [1233, 457, 555, 1912, 3499]
1343 [1250, 2324, 3030, 2731, 1207]
1344 [858, 2858,

1733 [1178, 527, 3030, 2571, 2329]
1734 [2028, 1221, 3730, 296, 1244]
1735 [2858, 3114, 1784, 2396, 1060]
1736 [1222, 1961, 924, 1960, 1957]
1737 [593, 1197, 1213, 2918, 1258]
1738 [589, 223, 1261, 3925, 1594]
1739 [1203, 3468, 608, 2997, 1296]
1741 [916, 541, 3307, 1945, 899]
1742 [1148, 745, 527, 1193, 1221]
1743 [922, 913, 2571, 260, 3114]
1744 [527, 912, 2028, 1272, 1276]
1745 [2762, 589, 2959, 2640, 2433]
1746 [1148, 1278, 3000, 678, 1304]
1747 [926, 1784, 150, 2071, 1200]
1748 [527, 1212, 913, 1198, 260]
1749 [1148, 593, 3030, 969, 2028]
1750 [908, 3578, 589, 3317, 3481]
1751 [3916, 3310, 3072, 480, 783]
1752 [745, 750, 1272, 2858, 1213]
1753 [1204, 1198, 2028, 905, 923]
1754 [1721, 3915, 1884, 2625]
1755 [926, 2804, 911, 1253, 1208]
1756 [3753, 2100, 2683, 3157]
1757 [898, 1252, 110, 356, 1307]
1758 [908, 1242, 3114, 1196, 58]
1760 [2125, 2243, 1965, 3422, 708]
1761 [541, 3468, 1214, 1301, 671]
1762 [1610, 3578, 1387, 6, 3256]
1763 [1257, 3948, 2746]
1764 [260, 1197, 1, 3507, 33

...

4386 [2019, 750, 2324, 1198, 1172]
4387 [745, 904, 50, 913, 858]
4388 [1250, 1193, 2571, 1262, 2858]
4389 [2804, 111, 1961, 3363, 2302]
4390 [2571, 356, 2542, 6, 2599]
4391 [2329, 150, 3105, 588, 1688]
4392 [1270, 1721, 2011, 2628, 2012]
4393 [2571, 104, 3298]
4394 [3005, 543, 2812, 2541, 2771]
4395 [527, 110, 589, 2294]
4396 [1299, 1610, 1079, 1265, 2023]
4397 [2762, 2959, 2948, 2333, 86]
4398 [608, 356, 2248, 1270, 3448]
4399 [1210, 595, 1097, 588, 2078]
4400 [356, 2125, 605]
4401 [1212, 913, 3030, 1198, 898]
4402 [953, 1219, 1997]
4403 [1148, 593, 2028, 1218, 110]
4404 [50, 2762, 2324, 2858, 110]
4405 [858, 898, 2858, 2804, 951]
4406 [2019, 904, 2762, 2571, 908]
4407 [2762, 1610, 2268, 778, 2580]
4408 [2019, 2731, 912, 1224, 1217]
4409 [1784, 223, 1095, 34, 2333]
4410 [3751, 1916, 3082, 2722]
4411 [1148, 593, 2324, 912, 953]
4412 [2571, 2858, 110, 2959, 2580]
4413 [2396, 1961, 3072, 2622, 2688]
4414 [2762, 2959, 539, 2359, 2355]
4415 [1242, 1233, 3265, 1263, 2396]
4416 [3000, 457, 9

4737 [1189, 2859, 1242, 1258, 1211]
4738 [318, 750, 2571, 908, 923]
4739 [318, 2028, 1278, 1136, 3114]
4740 [1961, 918, 39, 610, 1777]
4741 [904, 1213, 1256, 2073, 1918]
4742 [1207, 912, 908, 953, 954]
4743 [3468, 3095, 553, 3405, 1735]
4744 [969, 954, 1196, 2917, 2795]
4745 [527, 1193, 260, 1104, 1196]
4746 [1197, 2501, 2918, 1784, 534]
4747 [2762, 260, 1197, 1240, 2966]
4748 [904, 858, 953, 1234, 3114]
4749 [2628, 2861, 419]
4750 [2028, 2804, 1358, 1278, 2918]
4751 [3307, 1949, 3952, 3072, 1379]
4752 [1247, 1066, 933, 3668, 1571]
4753 [912, 2858, 1203, 971, 588]
4754 [1198, 1196, 1293, 111, 1258]
4755 [858, 986, 2023]
4756 [2912, 3174, 3155, 3081, 3055]
4757 [858, 1193, 1235, 608, 919]
4758 [36, 62, 3260, 1721]
4759 [1267, 608, 1188, 32, 1968]
4760 [541, 930, 1617, 1387, 1732]
4761 [296, 2997, 3471, 1682, 3052]
4762 [858, 1193, 908, 3307, 1234]
4763 [2858, 1267, 2359, 1912, 2628]
4764 [1307, 2795, 2130, 2081, 34]
4765 [1178, 913, 1250, 1204, 593]
4766 [912, 2858, 3147, 2959, 2908]
47

5158 [1204, 1193, 260, 2028, 1272]
5159 [919, 3513, 3190]
5160 [260, 2858, 608, 296, 778]
5161 [1250, 2571, 260, 2329, 1945]
5162 [733, 349, 2922, 2916, 2058]
5163 [913, 1945, 1272, 1278, 1234]
5164 [2028, 1224, 1272, 3147, 1233]
5165 [2762, 1207, 3022, 1224, 3307]
5166 [904, 527, 2028, 47, 608]
5167 [2762, 1136, 3147, 1253, 1617]
5168 [1196, 356, 1923, 62, 597]
5169 [110, 2396, 1307, 529, 1090]
5170 [908, 3429, 1358, 1234, 1449]
5171 [260, 1127, 3766]
5172 [318, 913, 1198, 912, 1172]
5173 [1923, 733, 2502, 34, 2355]
5174 [3007, 3129, 480, 3125, 3115]
5175 [3147, 110, 2889, 3752]
5176 [2028, 2502, 2355, 1516, 3286]
5177 [2324, 2571, 101, 555, 2359]
5178 [913, 3022, 910, 1234, 1136]
5179 [2731, 2571, 3000, 2761, 733]
5180 [1207, 908, 923, 954, 1196]
5181 [260, 296, 2396, 3408, 3481]
5182 [110, 1246, 150, 1257, 2268]
5183 [1136, 3114, 1784, 356, 2959]
5184 [2762, 1361, 2858, 47, 1230]
5185 [593, 969, 2804, 1784, 1263]
5186 [1253, 1301, 1921, 1298, 3527]
5187 [750, 1198, 2858, 608, 2396]


5556 [1193, 969, 58, 2692, 1240]
5557 [541, 1197, 1358, 1276, 1278]
5558 [2324, 1198, 2571, 2918, 1961]
5559 [110, 475, 1466, 534, 1747]
5560 [2571, 356, 2396, 924, 2080]
5561 [2028, 1361, 3114, 608, 2677]
5562 [2324, 1207, 912, 953, 3114]
5563 [527, 593, 1188, 1097]
5564 [2762, 3462, 2949, 1968, 3371]
5565 [913, 750, 2692, 589, 2997]
5566 [527, 1204, 1198, 260, 912]
5567 [527, 913, 2028, 905, 1172]
5568 [260, 899, 1214, 1036, 671]
5569 [745, 2804, 2918, 919, 1784]
5570 [260, 912, 1213, 1304, 2918]
5571 [750, 1193, 260, 930, 951]
5572 [527, 50, 945, 296, 3006]
5573 [904, 912, 1131, 1252, 3468]
5574 [260, 899, 1258, 919, 1617]
5575 [1276, 3114, 1230, 1281, 1292]
5576 [527, 2324, 541, 899, 914]
5577 [3634, 3713, 3706, 3649, 3697]
5578 [593, 2324, 858, 260, 1197]
5579 [318, 296, 1097, 2710]
5580 [318, 2762, 1250, 593, 1207]
5581 [1617, 1240, 349, 3256, 431]
5582 [913, 1262, 1224, 1945, 1203]
5583 [260, 110, 1283, 3578, 1953]
5584 [2609, 1214, 2949, 1321, 2716]
5585 [50, 1178, 2731, 1197, 

5896 [2324, 3408, 101, 440, 2390]
5897 [2908, 3481, 2721, 2076, 3301]
5898 [306, 2336, 1277, 2355, 2763]
5899 [1935, 597, 3489]
5900 [593, 858, 1225, 307, 150]
5901 [593, 1224, 1610, 538, 10]
5902 [750, 593, 2028, 1219, 914]
5903 [912, 110, 1, 2761, 223]
5905 [1193, 1258, 933, 1307, 1210]
5906 [593, 1193, 608, 1288, 2396]
5907 [50, 1212, 1213, 2208, 1784]
5908 [910, 1197, 1225, 1196, 110]
5909 [608, 1387, 1340, 1982, 1194]
5910 [527, 2918, 28, 1264, 2248]
5911 [913, 2324, 1198, 903, 2858]
5912 [904, 527, 1198, 1237, 1219]
5913 [318, 2324, 2692, 356, 497]
5914 [858, 2186, 1945, 1213, 1953]
5915 [318, 50, 593, 1197, 1221]
5916 [1178, 1212, 1193, 969, 2937]
5917 [2019, 318, 1250, 905, 1304]
5918 [608, 1961, 2791, 1270, 1997]
5919 [2572, 3081, 3053, 1911]
5920 [2762, 1198, 1224, 1358, 1196]
5921 [296, 589, 1090, 1200, 553]
5922 [318, 904, 527, 1250, 1284]
5923 [2762, 593, 1393, 3868, 2598]
5924 [2762, 1387, 2948, 2949, 349]
5925 [2731, 3469, 2203, 1197, 1252]
5926 [858, 1198, 1221, 457, 12

In [85]:
#Fraction of Concordant Pairs
accuracy.fcp(predictions, verbose=True)

FCP:  0.7199


0.71994673607749726

$n_{c}^u$ = |{(i,j) | {$\hat{r}_{ui}$ }> $\hat{r}_{uj}$ and r$_{ui}$ > r$_{uj}$}| 


Where $\hat{r}$ is the rating predictor.
$n_{c}^u$ is the concordant pairs. $n_{c}$ is the sum over all users.
In a similar way compute the discordant pairs $n_{d}^u$.

FCP = $\frac{n_{c}}{n_{c}+{n_{d}}}$